<a href="https://colab.research.google.com/github/lustraka/puzzles/blob/main/AoC2021/AoC_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Code Puzzles
[Advent of Code 2021](https://adventofcode.com/2021) | [reddit/adventofcode](https://www.reddit.com/r/adventofcode/)

In [ ]:
import requests
import pandas as pd
import numpy as np
path = 'https://raw.githubusercontent.com/lustraka/puzzles/main/AoC2021/data/'

## 